# SIngularity Containers

## What is a container?

A container is a standard unit of software that packages up code and all its dependencies so the application runs quickly and reliably from one computing environment to another. A container image is a lightweight, standalone, executable package of software that includes everything needed to run an application: code, runtime, system tools, system libraries and settings.

Containers use the host system's kernel, and can access its hardware more directly. When you run a process in a container it runs on the host system, directly visible as a process on that system. Unlike a Virtual Machine, container is a virtualization at the software level, whereas VMs are virtualization at hardware level. If you are interested in finding out more differences between VM and a container, go to this [link](https://www.electronicdesign.com/dev-tools/what-s-difference-between-containers-and-virtual-machines)

## Why use a container?

Containers package together a program and all of its dependencies, so that if you have the container you can use it on any Linux system with the container system software installed. It doesn't matter whether the system runs Ubuntu, RedHat or CentOS Linux - if the container system is available then the program runs identically on each, inside its container. This is great for distributing complex software with a lot of dependencies, and ensuring you can reproduce experiments exactly. If you still have the container you know you can reproduce your work. Also since the container runs as a process on the host machine, it can be run very easily in a [SLURM job](https://docs.uabgrid.uab.edu/wiki/Slurm)

## Docker vs Singularity?

[Docker](https://www.docker.com/) is the most popular and widely used container system in the industry. But [Singularity](https://www.sylabs.io/singularity/) was built keeping HPC in mind, i.e a shared environment. Singularity is designed so that you can use it within SLURM jobs and it does not violate security constraints on the cluster. Though, since Docker is very popular and a lot of people were already using the Docker for their softwares, Singularity maintained a compatibility for Docker images. We'll be seeing this compatibility later in the notebook. Both Singularity and Docker maintain a hub where you can keep your docker remotely, and pull them from anywhere. Here is a link for both the hubs:

[Singularity-Hub](https://singularity-hub.org)  
[Docker Hub](https://hub.docker.com/)


Singularity is already available on Cheaha. To check the available modules for Cheaha, run the cell below:


In [ ]:
!module avail Singularity

As you might have already noticed that we already loaded a Singularity module while starting up this notebook. You can check the version of the Singularity loaded below:

In [ ]:
!singularity --version

## Basic singularity command line functions:

To check the basic functions or command line options provided run help on the singularity 

In [ ]:
!singularity --help

To check more information about a particular parameter, use help in conjunction with that parameter

In [ ]:
!singularity pull help

## Example 1:

In this example we are going to be pulling images from [Singularity-Hub](https://singularity-hub.org/). This singularity image contains the tool [neurodebian](http://neuro.debian.net/).

NeuroDebian provides a large collection of popular neuroscience research software for the Debian operating system as well as Ubuntu and other derivatives.

Let's begin by pulling the [Neurodebian Singularity image](https://singularity-hub.org/collections/209) from Singularity Hub

In [ ]:
!singularity pull shub://neurodebian/neurodebian

Now that we have pulled the image you should be able to see that image in your directory by simply running a 'ls' command

In [ ]:
!ls

Now, we'll try to execute a command from within the container. Remember that exec parameter allows you to achieve this functionality. Let's list the content of you /data/user/$USER directory

In [ ]:
!singularity exec neurodebian-neurodebian-master-latest.simg ls /tmp

Hmmm, an error. Remember your singularity container image doesn't know about the directories on your host machine. It by default (in most containers) binds your HOME and tmp directory. So try to run the above command again but change the list path to your HOME directory.

Now, all our raw data is generally in our /data/user/$USER locations, so we really need to access that location if our container has to be useful. SIngularity provides you with a parameter (-B) to bind path from your host machine to the container. Try the same command again, but with the bind parameter

In [ ]:
!singularity exec -B /data neurodebian-neurodebian-master-latest.simg ls /data/user/wsmonroe

Now rather then using ls command from within the container we are going to see one of the softwares within the container: [dcm2nii](https://www.nitrc.org/projects/mricron)

In [ ]:
!singularity exec -B /data neurodebian-neurodebian-master-latest.simg dcm2nii

## Example 2:

In this example we are going to be pulling a singularity image from [dockerhub](https://hub.docker.com/). This singularity image contains [google-cloud-sdk tools](https://cloud.google.com/sdk/).

The Cloud SDK is a set of tools for Cloud Platform. It contains gcloud, gsutil, and bq command-line tools, which you can use to access Google Compute Engine, Google Cloud Storage, Google BigQuery, and other products and services from the command-line. You can run these tools interactively or in your automated scripts.

In [ ]:
!singularity pull docker://jess/gcloud

In [ ]:
!singularity exec -B /data gcloud.simg gsutil

To clean your directory of all the container images, you can run the command below

In [ ]:
!rm *.simg

## Example 3:

[NVIDIA GPU Cloud](https://www.nvidia.com/en-us/gpu-cloud/) (NGC) offers a container registry of Docker images with over 35 HPC, HPC visualization, deep learning, and data analytics containers optimized for GPUs and delivering accelerated performance (figure 3). The registry includes some of the most popular applications including GROMACS, NAMD, ParaView, VMD, and TensorFlow.

For this example you would have to start a new Jupyter Notebook session on the 'pascalnodes' partition. Most of the settings are similar to the ones that you had in the [Git repo](https://gitlab.rc.uab.edu/rc-training-sessions/singularity_containers) for this session. you just need to add/modify the following things.

In the Environment parameter add
```
module load cuda92/toolkit/9.2.88
module load CUDA/9.2.88-GCC-7.3.0-2.30
module load Singularity/2.5.2-GCC-5.4.0-2.26
module load Anaconda3
```

And in the partition choose:
```
pascalnodes
```

You will need to create a new account on Nvidia GPU Cloud, to pull down these conatiners.

In [ ]:
!singularity pull docker://nvcr.io/hpc/vmd:cuda9-ubuntu1604-egl-1.9.4a17

In [ ]:
!singularity exec --nv vmd-cuda9-ubuntu1604-egl-1.9.4a17.simg /opt/vmd/bin/vmd

## Building a Singularity container recipe file

Though building a recipe file for Singularity containers is beyond the scope of this session, we have provided a few important links below which would tell you how to create a recipe file for Singularity containers as well as build the container using them.

When you want to create a container for production use on the cluster, you should build a container image from a definition file. Unfortunately, building containers from a definition file requires you to be a system administrator (root) on the machine you use for building. You will need to build Singularity containers on a machine that you control.

To install Singularity on your system, follow the steps outlined here:  
http://singularity.lbl.gov/install-linux  
http://singularity.lbl.gov/install-mac  
http://singularity.lbl.gov/install-windows  

Method of creating a Singularity recipe file:  
http://singularity.lbl.gov/docs-recipes

Method of building from a singularity recipe file:  
http://singularity.lbl.gov/docs-build-container#building-containers-from-singularity-recipe-files




## Acknowledgments / Useful Links:

https://github.com/singularityhub/singularityhub.github.io/wiki  
https://portal.biohpc.swmed.edu/content/guides/singularity-containers-biohpc/  
https://www.docker.com/  
https://devblogs.nvidia.com/docker-compatibility-singularity-hpc/  